In [2]:
import numpy as np
import torch
import os
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
import time
from tqdm import tqdm

In [3]:
import os
path = 'NewPlantDiease/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)'
train_data1 = path + '/train'
val_data1 = path + '/valid'
diease = os.listdir(train_data1)

In [11]:
mean = [0.5, 0.5, 0.5]
std = [1, 1, 1]
transforms = transform.Compose([transform.Resize((256, 256)), transform.ToTensor()])
train_datasets = ImageFolder(root = train_data1, transform=transforms)
val_datasets = ImageFolder(root = val_data1, transform=transforms)
Train_dataloader = DataLoader(dataset = train_datasets,  batch_size= 512, shuffle = False)
Val_dataloader = DataLoader(dataset = val_datasets,  batch_size= 256, shuffle = False )

In [12]:
LEN_TRAIN = len(train_datasets)
LEN_val = len(val_datasets)
print(LEN_TRAIN, LEN_val)

70295 17572


In [15]:
image , label = next(iter(Train_dataloader))
image[0]

tensor([[[0.6549, 0.6510, 0.6471,  ..., 0.7255, 0.7176, 0.7098],
         [0.6667, 0.6627, 0.6588,  ..., 0.7294, 0.7216, 0.7176],
         [0.6667, 0.6627, 0.6588,  ..., 0.7373, 0.7294, 0.7255],
         ...,
         [0.3922, 0.4000, 0.4078,  ..., 0.4941, 0.4941, 0.4941],
         [0.3961, 0.4000, 0.4039,  ..., 0.4784, 0.4824, 0.4824],
         [0.4078, 0.4039, 0.4000,  ..., 0.4745, 0.4784, 0.4824]],

        [[0.6392, 0.6353, 0.6314,  ..., 0.7137, 0.7059, 0.6980],
         [0.6510, 0.6471, 0.6431,  ..., 0.7176, 0.7098, 0.7059],
         [0.6510, 0.6471, 0.6431,  ..., 0.7255, 0.7176, 0.7137],
         ...,
         [0.3529, 0.3608, 0.3686,  ..., 0.4314, 0.4314, 0.4314],
         [0.3569, 0.3608, 0.3647,  ..., 0.4157, 0.4196, 0.4196],
         [0.3686, 0.3647, 0.3608,  ..., 0.4118, 0.4157, 0.4196]],

        [[0.7686, 0.7647, 0.7608,  ..., 0.8275, 0.8196, 0.8118],
         [0.7804, 0.7765, 0.7725,  ..., 0.8314, 0.8235, 0.8196],
         [0.7804, 0.7765, 0.7725,  ..., 0.8392, 0.8314, 0.

In [14]:
# calculating mean and STD
import numpy as np
s1 = torch.tensor([0.0, 0.0, 0.0])
s2 = torch.tensor([0.0, 0.0, 0.0])
for image,_ in tqdm(Train_dataloader):
    # print(image.size())
    s1 += image.sum(axis=[0,2,3])
    s2 += (image**2).sum(axis=[0,2,3])
n = LEN_TRAIN * image.size(2) * image.size(3)

mean = s1 / n
std = np.sqrt((s2/n) - ((s1/n)**2))
mean, std

100%|██████████| 138/138 [02:26<00:00,  1.06s/it]


(tensor([0.4757, 0.5001, 0.4264]), tensor([0.2165, 0.1957, 0.2320]))

In [4]:
LEN_TRAIN = len(train_datasets)
LEN_val = len(val_datasets)
print(LEN_TRAIN, LEN_val)

70295 17572


In [5]:
# images, labels = next(iter(dataloader))
# print(images.size())
# # Access the first image and label
# first_image = images[2]
# first_label = labels[0]

In [6]:
# print(first_image)

In [5]:
from torchvision import models
import torch.nn as nn
model = models.resnet18(pretrained=True)
# model
# Freeze all parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = False
    
# # Extracting features from last CNN layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features , 38)
# model

/home/.conda/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/.conda/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# optimizer and loss function
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr = 1e-4)
loss_function = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(50):
    start = time.time()
    
    tr_acc = 0
    val_acc = 0
    
    # Train
    model.train()
    
    with tqdm(Train_dataloader, unit="batch") as tepoch:
        for xtrain, ytrain in tepoch:
            
            train_prob = model(xtrain)
        
            
            train_loss = loss_function(train_prob, ytrain)
            train_loss.backward() # backpropagation

            # loss.backward()
            optimizer.step() # update the model parameters
            optimizer.zero_grad()
            # training ends
            
            train_pred = torch.max(train_prob, 1).indices
            tr_acc += int(torch.sum(train_pred == ytrain))
            
        ep_tr_acc = tr_acc / LEN_TRAIN
    end = time.time()
    duration = (end - start) / 60
    
    # print(f"Epoch: {epoch}, Test_Loss: {train_loss}, Train_acc: {ep_tr_acc}")
    # Evaluate
    
#     model.eval()
#     # with torch.no_grad():
    with torch.no_grad():
        for xval, yval in Val_dataloader:
            val_prob = model(xval)

            
            val_pred = torch.max(val_prob,1).indices
            val_acc += int(torch.sum(val_pred == yval))
            
        ep_val_acc = val_acc / LEN_val
    
    end = time.time()
    duration = (end - start) / 60
    
    print(f"Epoch: {epoch}, Test_Loss: {train_loss}, Train_acc: {ep_tr_acc}, Val_acc: {ep_val_acc}")
    

  4%|▍         | 12/275 [00:56<20:24,  4.65s/batch]

In [7]:
for x,y in Train_dataloader:
    y_prob = model(x)
    break
    

In [36]:
# y_prob

In [29]:
pred = torch.max(y_prob,1).indices
pred

tensor([22, 21, 31, 15, 30, 21, 31, 21, 22, 21, 21, 22, 15, 30, 31, 31])

In [25]:
y

tensor([30, 23, 17, 13, 16, 37, 31, 36,  1, 37, 24, 35, 21,  3,  2,  4])

In [22]:
y_prob.size()

torch.Size([16, 38])

In [30]:
int(torch.sum(pred == y))

1

In [ ]:
# # Training
# for epoch in range(num_epochs):
#     # Number of epochs,
#     for i, (images, labels) in enumerate(train_loader):

# # iterate over data
# # measure accuarcy 
# # measure loss

In [ ]:
# pixel = 0
# s1 = 0
# s2 = 0
# for image, _ in dataloader:
#     n = images.size(2) * images.size(3)
#     # print(n)
#     s1 += n
#     s2 += s1**2
# print(np.sqrt((s2/n) - (s1/n)**2))

In [39]:
s1 = np.zeros(3)
s2 = np.zeros(3)
s1,s2



(array([0., 0., 0.]), array([0., 0., 0.]))

# EarlyStopping

In [4]:
mean = [0.4757, 0.5001, 0.4264]
std = [0.2165, 0.1957, 0.2320]
transforms = transform.Compose([transform.Resize((256, 256)), transform.ToTensor(), transform.Normalize(mean=mean, std=std)])
train_datasets = ImageFolder(root = train_data1, transform=transforms)
Len_TRAIN = len(train_datasets)
train_data = int(0.95 * Len_TRAIN)
testing_data = Len_TRAIN - train_data
train_set, val_set = torch.utils.data.random_split(train_datasets, [train_data, testing_data])
Train_dataloader = DataLoader(dataset = val_set,  batch_size= 256, shuffle = True )
LEN_TRAIN = len(val_set)
len(Train_dataloader), len(val_set),  LEN_TRAIN

(14, 3515, 3515)

In [5]:
val_datasets = ImageFolder(root = val_data1, transform=transforms)
Len_val = len(val_datasets)
train_data1 = int(0.9 * Len_val)
testing_data1 = len(val_datasets) - train_data1
train_set1, val_set1 = torch.utils.data.random_split(val_datasets, [train_data1, testing_data1])
Val_dataloader = DataLoader(dataset = val_set1,  batch_size= 256, shuffle = False )
LEN_val = len(val_set1)
validation_set1 = val_set1.dataset
len(validation_set1.classes),len(val_set1), LEN_val

(38, 1758, 1758)

In [6]:
from torchvision import models
import torch.nn as nn
model = models.resnet18(pretrained=True)
# model
# Freeze all parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = True
# # Extracting features from last CNN layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features , 38)

/home/.conda/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/.conda/envs/pytorch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
class EarlyStopping:
    def __init__(self, patience):
        self.pateince  = patience
        self.best_score = None
        self.counter = None
        self.stop  = False
    def _stopping(self,val_loss):
        if self.best_score is None:
            self.best_score = val_loss
        elif val_loss > self.best_score:
            self.counter += 1
            if self.counter == self.pateince:
                self.stop = True
        else:
            self.best_score = val_loss
            print(self.best_score)
            counter = 0

In [8]:
# optimizer and loss function
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr = 1e-5)
loss_function = nn.CrossEntropyLoss()

In [ ]:
earlystopping = EarlyStopping(patience=3)
for epoch in range(50):
    start = time.time()
    
    tr_acc = 0
    val_acc = 0
    
    # Train
    model.train()
    
    with tqdm(Train_dataloader, unit="batch") as tepoch:
        for xtrain, ytrain in tepoch:
            
            train_prob = model(xtrain)
        
            
            train_loss = loss_function(train_prob, ytrain)
            train_loss.backward() # backpropagation

            # loss.backward()
            optimizer.step() # update the model parameters
            optimizer.zero_grad()
            # training ends
            
            train_pred = torch.max(train_prob, 1).indices
            tr_acc += int(torch.sum(train_pred == ytrain))
            
        ep_tr_acc = tr_acc / LEN_TRAIN
    end = time.time()
    duration = (end - start) / 60
    
    # print(f"Epoch: {epoch}, Test_Loss: {train_loss}, Train_acc: {ep_tr_acc}")
    # Evaluate
    
    model.eval()
#     # with torch.no_grad():
    with torch.no_grad():
        for xval, yval in Val_dataloader:
            val_prob = model(xval)
            
            val_loss = loss_function(val_prob , yval)
            
            val_pred = torch.max(val_prob,1).indices
            val_acc += int(torch.sum(val_pred == yval))
            
        ep_val_acc = val_acc / LEN_val
    avg_val_loss = val_loss / len(Val_dataloader)
    earlystopping._stopping(val_loss)
    if earlystopping.stop:
        print('Early Stopping Excuated')
        break
    end = time.time()
    duration = (end - start) / 60
    
    print(f"Epoch: {epoch}, Test_Loss: {train_loss},Validation_loss: {val_loss}, Train_acc: {ep_tr_acc}, Val_acc: {ep_val_acc}")
    

100%|██████████| 14/14 [02:42<00:00, 11.58s/batch]


Epoch: 0, Test_Loss: 3.6161789894104004,Validation_loss: 3.6524455547332764, Train_acc: 0.030440967283072547, Val_acc: 0.03185437997724687


100%|██████████| 14/14 [02:39<00:00, 11.39s/batch]


tensor(3.3905)
Epoch: 1, Test_Loss: 3.333961248397827,Validation_loss: 3.390486001968384, Train_acc: 0.06770981507823613, Val_acc: 0.0813424345847554


100%|██████████| 14/14 [02:40<00:00, 11.44s/batch]


...

100%|██████████| 14/14 [02:38<00:00, 11.29s/batch]


tensor(1.3925)
Epoch: 11, Test_Loss: 1.1881040334701538,Validation_loss: 1.3924964666366577, Train_acc: 0.8867709815078236, Val_acc: 0.8293515358361775


100%|██████████| 14/14 [02:37<00:00, 11.25s/batch]


tensor(1.2816)
Epoch: 12, Test_Loss: 1.0882892608642578,Validation_loss: 1.2815687656402588, Train_acc: 0.9046941678520626, Val_acc: 0.8481228668941979


100%|██████████| 14/14 [02:38<00:00, 11.36s/batch]


0
1
2
3
4
5
6
7
8
9
